In [15]:
import os
import pandas
import plotly.express as px
import plotly.graph_objects as go

In [16]:
def extract_generation_from_filename(filename):
    generation = filename.split('_')[1][:2]
    return int(generation) if generation[1].isdigit() else int(generation[0])

In [17]:
def read_report_data(data_directory):
    dataframes = {}
    for filename in os.listdir(data_directory):
        if filename.endswith(".csv"):
            path = os.path.join(data_directory, filename)
            df = pandas.read_csv(path)
            generation = extract_generation_from_filename(filename)
            df['generation'] = [generation for _ in range(0,len(df.index))]
            dataframes[filename] = df
    return dataframes

In [18]:
data_directory = '../data/genetic/2020-10-14_11-49-58'
dataframes = read_report_data(data_directory)

In [19]:
for filename, df in dataframes.items():
    df.drop(df.columns.difference(['total_net_return', 'generation', 'average_trades_per_month', 'max_draw_down']), 1, inplace=True)
    
df = pandas.concat([df for filename, df in dataframes.items()])

In [20]:
df

,average_trades_per_month,max_draw_down,total_net_return,generation
0,1.666667,0.072486,112.734785,7
1,2.333333,0.125078,108.081949,7
2,1.800000,0.348026,2.992490,7
3,2.500000,0.051527,87.976558,7
4,2.500000,0.053177,78.010845,7
...,...,...,...,...
45,9.571429,1.011975,-87.545959,18
46,9.571429,1.011975,-87.545959,18
47,12.428571,0.578566,-41.594348,18
48,12.428571,0.820025,-70.486029,18


python -m backtesting_engine --csv Kraken-BTC-EUR:testdata/Bitstamp-BTC-USD_010120-010820_m/Kraken-BTC-EUR.csv --optimize "total_net_return_percent:0.3" "max_draw_down:0.9" "average_trades_per_month:0.9" --mutation-rate 0.4 --generations 50 --population-size 50 --elitism-rate 0.5 --eta 6 --working-capital 100 Momentum --market Kraken-BTC-EUR --amount-quote 100 --open-long mutable:0.05:0.25 --open-short mutable:-0.25:-0.05 --trailing-stop-loss-long mutable:-0.25:-0.05 --trailing-stop-loss-short mutable:-0.25:-0.05 --max-volatility 1

In [22]:
fig = px.box(df, x='generation', y="max_draw_down")
fig.write_html('max_dd_over_generations.html')
fig.show()
# Zeile 21 in ETA-Results